## 1. ARIMA 모델링 – 쉽게 풀어보기

1. **ARIMA란?**  
   - **AR**(AutoRegressive): 과거 값들이 현재 값에 영향을 준다고 가정  
   - **I**(Integrated): 차분(differencing)으로 시계열을 안정화  
   - **MA**(Moving Average): 과거 예측 오차들이 현재에 반영됨  

2. **(1,1,1) 의미**  
   - **p=1**: 바로 전날 가격 하나만 자기회귀(AR)로 사용  
   - **d=1**: ‘하루 차분’ → `today − yesterday` 값을 예측 대상으로 삼아 시계열을 정상(stationary)으로 만듦  
   - **q=1**: 가장 최근 예측 오차 하나(MA)를 모델에 포함  

  ### **2-1. 차분의 개념**  
- **차분(1차 차분)**이란 “오늘값 – 어제값”을 계산하는 것  
- 이렇게 하면 **추세가 제거**되어,  
  - 원래 데이터가 꾸준히 상승했다면  
  - 차분된 데이터는 오르내림(증가량)만 남아 평탄해집니다.

---

  ### 2-2. **간단한 숫자 예시**

  | t (일) | 원본 가격 \(y_t\) | 1차 차분 \(y_t - y_{t-1}\) |
  |-------|------------------|-----------------------------|
  | 1     | 10               | –                           |
  | 2     | 12               | \(12 - 10 = 2\)             |
  | 3     | 15               | \(15 - 12 = 3\)             |
  | 4     | 20               | \(20 - 15 = 5\)             |
  | 5     | 27               | \(27 - 20 = 7\)             |

- **원본**: 10 → 12 → 15 → 20 → 27 (계속 증가하는 추세)  
- **차분**: 2, 3, 5, 7 (증가량만 남음)  
  - 이 차분 시퀀스는 더 이상 “꾸준히 오르는” 추세가 없고, **평균 변화량** 주위로 움직입니다.


3. **학습 & 예측 흐름**  
   - 훈련: 과거 3개월 − 마지막 20일 구간까지  
   - 모델은 “차분된 시계열”을 AR(1) + MA(1) 공식으로 학습  
   - 예측: 남은 20일(테스트 구간)에 대해, 하루씩 차분된 예측값을 원래 수준으로 복원해 출력  

4. **장·단점 요약**  
   - 장점: 시계열 특성(추세·자기상관·노이즈) 직접 모델링  
   - 단점: 비선형 패턴이나 외부 변수(거래량·뉴스)는 반영 못 함  

---

## 2. XGBoost로 예측하기 – 단계별 설명

1. **왜 XGBoost?**  
   - 트리 기반 앙상블 기법으로,  
   - 비선형 관계나 복잡한 상호작용을 잘 잡아냅니다.

2. **Lag & Rolling 피처(Feature) 만들기**  
   - `lag1`~`lag5`: 1 ~ 5일 전 종가를 새로운 컬럼으로 추가  
     - “어제·2일 전·…·5일 전” 가격이 오늘 가격에 주는 영향을 학습  
   - `roll_mean5`: 과거 5일 종가 평균  
     - 최근 5일의 ‘전반적 흐름’을 모델에 알려 줌  

3. **학습 & 예측 과정**  
   - 훈련: lag/rolling 특성과 종가를 이용해 XGBoost 회귀 모델 학습  
   - 예측: 테스트 20일 구간의 피처를 입력 → `predict()`로 종가 반환  

4. **주요 하이퍼파라미터**  
   - `n_estimators=100`: 트리 100개 사용  
   - `learning_rate=0.1`: 각 트리가 기여하는 정도  
   - **돋보기**: 트리 수나 학습률을 늘리면 → 과적합↑/속도↓, 줄이면 → 과소적합↑/속도↑  

5. **장·단점**  
   - 장점: 외부 변수·비선형성 반영, 강력한 예측 성능  
   - 단점: 시계열 순서를 직접 반영하지 않으니 lag 같은 전처리가 필수  

---


In [ ]:
import pandas as pd
import yfinance as yf

# 1) 삼성전자(005930.KS) 최근 3개월 일별 주가 가져오기
ticker = '005930.KS'
data = yf.download(ticker, period='3mo', interval='1d')
data = data[['Open','High','Low','Close','Volume']]
data.to_csv('/content/drive/MyDrive/AI/Data/samsung_3mo.csv')

# 2) ARIMA 모델링
from statsmodels.tsa.statespace.sarimax import SARIMAX
# 종가만 사용
ts = data['Close']
train_ts = ts.iloc[:-20]
test_ts = ts.iloc[-20:]

# 단순 SARIMAX(1,1,1)
model_arima = SARIMAX(train_ts, order=(1,1,1), enforce_stationarity=False, enforce_invertibility=False)
res_arima = model_arima.fit(disp=False)
n_train = len(train_ts)
n_test  = len(test_ts)
pred_arima = res_arima.predict(start=n_train, end=n_train+n_test-1, typ='levels')
print(pred_arima)


# 3) XGBoost 모델링
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# Feature engineering: lag features
df_xgb = data[['Close']].copy()
for lag in range(1,6):
    df_xgb[f'lag{lag}'] = df_xgb['Close'].shift(lag)
df_xgb['roll_mean5'] = df_xgb['Close'].rolling(5).mean()
df_xgb.dropna(inplace=True)

X = df_xgb.drop('Close', axis=1)
y = df_xgb['Close']
# split
train_X, test_X = X.iloc[:-20], X.iloc[-20:]
train_y, test_y = y.iloc[:-20], y.iloc[-20:]

xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=0)
xgb_model.fit(train_X, train_y)
pred_xgb = xgb_model.predict(test_X)
print(pred_xgb)

In [ ]:
import numpy as np
import itertools
import statsmodels.api as sm

p = q = range(0, 4)
d = [0, 1]
best_aic = np.inf
best_order = None

for order in itertools.product(p, d, q):
  try:
    model = sm.tsa.SARIMAX(ts, order=order)
    res = model.fit(disp=False)
    if res.aic < best_aic:
      best_aic = res.aic
      best_order = order
  except:
    continue

print("Best ARIMA order:", best_order, "AIC:", best_aic)

In [ ]:
from pmdarima import auto_arima
auto_model = auto_arima(ts, seasonal=False, stepwise=True,
                        max_p=5, max_q=5, d=None)
print(auto_model.order)